# 1. ETL: Extraccion de datos y seleccion de casos por grupos

## 1.1. Fuente de datos: Coingecko
Nuestra fuente de datos es la plataforma de CoinGecko. En esta seccion examinamos la disponibilidad de informacion en dicha plataforma.

Segun las consultas de abajo, la plataforma CoinGecko provee informacion para una gran cantidad de criptomonedas y, adicionalmente, con una amplia diversidad segun sus caracteristicas tecnologicas y de uso.

In [1]:
# manejo archivos
import os

# extraccion datos: API coingecko
import time
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI() # instanciamos

# manejo datos
import statistics as st
import pandas as pd
import json
from datetime import date

# extraccion datos deflactores de precios: API banco mundial
import wbgapi as wb
from collections.abc import Iterable

 Primero, esta plataforma tiene datos para mas de 10.000 criptomonedas.

In [2]:
# universo de monedas
coins_list = cg.get_coins_list()
data_coins_list = pd.DataFrame(coins_list).rename(columns={'id':'coin_id'})
print("Universo de monedas en coingecko:",
    data_coins_list['coin_id'].nunique())
#data_coins_list.info()
data_coins_list.head(3)

Universo de monedas en coingecko: 10214


,coin_id,symbol,name
0,01coin,zoc,01coin
1,0chain,zcn,Zus
2,0vix-protocol,vix,0VIX Protocol


Segundo, la plataforma tambien provee alrededor de 200 etiquetas sobre distintas caracteristicas de cada criptomonedas. A su vez, estas caracteristicas no son excluyentes; es decir, cada criptomonedas puede posser varias caracteristicas a la vez.

In [3]:
categories_list = cg.get_coins_categories_list()
categories_list = pd.DataFrame(categories_list)#.rename(columns={'id':'coin_id'})
print("Universo de categorias en coingecko:",
    categories_list['category_id'].nunique())
#categories_list.info()
print(categories_list.head(15))

# exportamos para inspeccion visual
categories_list.to_csv(os.path.join("0_reftables","categories_list.csv"))

Universo de categorias en coingecko: 209
                category_id                          name
0               aave-tokens                   Aave Tokens
1        algorand-ecosystem            Algorand Ecosystem
2    alleged-sec-securities        Alleged SEC Securities
3                 analytics                     Analytics
4             animal-racing                 Animal Racing
5            animoca-brands                Animoca Brands
6           aptos-ecosystem               Aptos Ecosystem
7        arbitrum-ecosystem            Arbitrum Ecosystem
8   arbitrum-nova-ecosystem       Arbitrum Nova Ecosystem
9                       art                           Art
10     art-blocks-ecosystem          Art Blocks Ecosystem
11  artificial-intelligence  Artificial Intelligence (AI)
12      asset-backed-tokens           Asset-backed Tokens
13            asset-manager                 Asset Manager
14        augmented-reality             Augmented Reality


## 1.2. Seleccion de casos: Criptomonedas

En esta seccion, extraemos datos sobre los casos de interés. Esta seleccion la hacemos con base en dos criterios relacionados con caracteristicas tecnologicas y tambien con base en su desempeno en el mercado. Con respecto a sus caracteristicas tecnologicas, nos concentramos en aquellos casos que evidencian autonomia y que  su valor esten vinculado a un activo tranzable, preferiblemente el USD. Con respecto a su comportamiento en el mercado, nos concentramos en aquellos casos que evidencia un alto volumen de intercambio. De esta manera, la seleccion de casos es la interseccion de estas dos caracteristicas.

Con respecto a las caracteristicas tecnologicas de la moneda, hacemos uso de categorias pre-establecidas en CoinGecko. Como mencionamos anteriormente, no existe un consenso claro con respecto a una classificación jerárquica de las criptomonedas sino mas bien una diversidad de caracteristicas tecnologicas. Dado nuestro interes en revelar XXX, consideramos que las caracteristicas tecnologicas mas importantes son su grado de autonomia y si su valor estan vinculado a un activo tranzable. Consideramos que las categorias `layer-1` y `stablecoins` de CoinGecko son equivalentes a estos conceptos, respectivamente. Por tal motivo, nos enfocamos unicamente en las criptomonedas que cumplen con estas caracteristicas tecnologicas: `layer-1` y `stablecoins`.

Con respecto a su comportamiento en el mercado, nos concentramos en el subconjunto de 10 monedas que evidencian el mayor volumen tranzado por categoria. Como mencionamos anteriormente, el caso mas representativo de los clientes de nuestra organizacion son cabeza de hogar de clase media con baja disposicion al riesgo. Por tanto, el volumen de intercambio orienta nuestra seleccion de casos, con el fin de garantizar un mercado amplio para su eventual liquidacion.

In [4]:
# seleccionamos las categorias de interes
filter_category_id = [
    'layer-1',
    'stablecoins'
    ]
categories_final = categories_list.loc[categories_list['category_id'].isin(filter_category_id)]
categories_final.head()

,category_id,name
105,layer-1,Layer 1 (L1)
174,stablecoins,Stablecoins


### 1.2.1. Disponibilidad de informacion de casos por categoria y comportamiento en el mercado

In [5]:
# Parametros globales

# divisa en la cual se expresa el valor de cada criptomoneda
param_vs_currency="usd"

# criterio de para ordenar los resultados
# en nuestro caso, seleccionamos volume_desc
# que corresponde al volumen tranzado en orden inverso
param_order="market_cap_desc"

# coingecko nos permite extraer informacion sobre los casos
# segun un criterio para ordenar los resultados que dependen de la fecha actual de la consulta
# seleccionamos los primeros diez casos segun el volumen tranzado en orden inverso; es decir,
# seleccionamos los 10 casos con el mayor volumen tranzado al momento de la consulta
# incluimos una columna fecha_consulta en el df final
# puesto que de esta fecha depende nuestro criterio de inclusion de casos
num_casos=20

# df vacio
data_coins_markets = pd.DataFrame()

# loop para extraer datos por cada categoria
for cat in list(categories_final['category_id'].unique()):
    param_category=cat
    coins_markets = cg.get_coins_markets(
        vs_currency=param_vs_currency,
        category=param_category,
        order=param_order
    )
    data_coins_markets_loop = pd.DataFrame(coins_markets).rename(columns={'id':'coin_id'})
    data_coins_markets_loop['category_id']=cat
    
    # seleccionamos las columnas de interes
    data_coins_markets_loop_vol=data_coins_markets_loop.iloc[:num_casos][
        ['category_id','coin_id','symbol','name','market_cap_rank','total_volume']
        ]
    
    # unimos
    data_coins_markets=pd.concat([data_coins_markets,data_coins_markets_loop_vol])
# movemos 'category_id' a la primera fila para facilitar la visualizacion
first_column=data_coins_markets.pop('category_id')
data_coins_markets.insert(0, 'category_id', first_column)

# df final

data_coins_markets['fecha_consulta'] = date.today()
data_coins_markets.info()
data_coins_markets.head()

<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, 0 to 19
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   category_id      40 non-null     object 
 1   coin_id          40 non-null     object 
 2   symbol           40 non-null     object 
 3   name             40 non-null     object 
 4   market_cap_rank  40 non-null     float64
 5   total_volume     40 non-null     float64
 6   fecha_consulta   40 non-null     object 
dtypes: float64(2), object(5)
memory usage: 2.5+ KB


,category_id,coin_id,symbol,name,market_cap_rank,total_volume,fecha_consulta
0,layer-1,bitcoin,btc,Bitcoin,1.0,9.148523e+09,2023-08-25
1,layer-1,ethereum,eth,Ethereum,2.0,7.586720e+09,2023-08-25
2,layer-1,binancecoin,bnb,BNB,4.0,3.816019e+08,2023-08-25
3,layer-1,cardano,ada,Cardano,8.0,2.197924e+08,2023-08-25
4,layer-1,solana,sol,Solana,10.0,2.863905e+08,2023-08-25


In [6]:
print("Numero total de casos:",
    data_coins_markets['coin_id'].nunique())
print("Numero total de casos tipo 'layer-1'",
    data_coins_markets['coin_id'].loc[
        data_coins_markets['category_id'].str.contains('layer-1')].nunique())
print("Numero total de casos tipo 'stablecoins':",
    data_coins_markets['coin_id'].loc[
        data_coins_markets['category_id'].str.contains('stablecoins')].nunique())
print("Lista de casos:",
    data_coins_markets['coin_id'].unique())

Numero total de casos: 40
Numero total de casos tipo 'layer-1' 20
Numero total de casos tipo 'stablecoins': 20
Lista de casos: ['bitcoin' 'ethereum' 'binancecoin' 'cardano' 'solana' 'polkadot'
 'the-open-network' 'bitcoin-cash' 'avalanche-2' 'cosmos'
 'hedera-hashgraph' 'crypto-com-chain' 'near' 'kaspa' 'algorand' 'tezos'
 'elrond-erd-2' 'injective-protocol' 'fantom' 'radix' 'tether' 'usd-coin'
 'dai' 'binance-usd' 'true-usd' 'frax' 'usdd' 'paxos-standard' 'pax-gold'
 'tether-gold' 'first-digital-usd' 'gemini-dollar' 'liquity-usd'
 'tether-eurt' 'eusd-27a558b0-8b5b-4225-a614-63539da936f4' 'alchemix-usd'
 'stasis-eurs' 'terrausd' 'bilira' 'bob']


### 1.2.2. Disponibilidad de informacion cualitativa por categoria
Hacemos uso del endpoint `cg.get_coin_by_id()`, el cual arroja una diversidad de indicadores y caracteristicas para cada criptomoneda. Nos concentramos unicamente en la columna categories; esa columna arroja una lista con todas las etiquetas asignadas a cada criptomoneda. De esta manera podemos explorar otras caracteristicas de nuestro universo de casos seleccionados para entender un poco sobre su contexto tecnologico.

In [7]:
# global df
data_coins_info=pd.DataFrame()

# subloop df
data_coins_id=pd.DataFrame()

# las categorias de nuestro interes: ['layer-1', 'stablecoins']
list_category = list(data_coins_markets['category_id'].unique())

for cat in list_category:
    
    list_cat_coins=list(
        data_coins_markets['coin_id'].loc[
            data_coins_markets['category_id'].str.contains(cat)
            ].unique())
    
    for coin_id in list_cat_coins:
        # coingecko tiene una limite maximo de 30 consultas por min.;
        # introducimos una pausa de 2.1 segundos para no superar el limite:
        # 60/2.1 = 28.57 consultas por min.
        time.sleep(2.1)
        
        # consulta
        param_coin_id = coin_id
        param_localization = False
        consulta_coinbyid = cg.get_coin_by_id(
            id=param_coin_id,
            localization=param_localization)
    
        # Extraemos data en formato JSON
        data_coins_id_loop = pd.DataFrame()
        data_coins_id_loop = pd.DataFrame.from_dict(
            consulta_coinbyid, orient='index').reset_index().rename(
                columns={'index':'variables',0:'value'})
        
        # hacemos uso de pivot para transponer los datos
        # de una formato largo a uno ancho.
        data_coins_id_loop_pvt = pd.DataFrame()
        data_coins_id_loop_pvt = data_coins_id_loop.pivot_table(
            columns='variables',
            values='value',
            aggfunc=list)

        # como las columnas heredaron su formato lista de listas
        # hacemos uso de explode para desanidar
        data_coins_id_loop_pvt_explt = pd.DataFrame()
        data_coins_id_loop_pvt_explt=data_coins_id_loop_pvt.explode(
            list(data_coins_id_loop_pvt.columns)).rename(columns={'id':'coin_id'})
        
        # seleccionamos unicamente las columnas de interes
        data_coins_id_loop_pvt_explt_2join = pd.DataFrame()
        data_coins_id_loop_pvt_explt_2join = data_coins_id_loop_pvt_explt[
            ['coin_id','categories']].copy()
        
        # asignamos la categoria del loop
        data_coins_id_loop_pvt_explt_2join['category_id'] = cat
        
        # combinamos dfs
        data_coins_id=pd.concat([data_coins_id,data_coins_id_loop_pvt_explt_2join])
        
    # unimos los dfs
    data_coins_info=pd.concat([data_coins_info,data_coins_id])

# movemos 'category_id' a la primera columna para facilitar visualizacion
first_column=data_coins_info.pop('category_id')
data_coins_info.insert(0,'category_id',first_column)
data_coins_info.drop_duplicates(subset=['category_id','coin_id'], inplace=True)
data_coins_info.head(3)

variables,category_id,coin_id,categories
value,layer-1,bitcoin,"[Cryptocurrency, Layer 1 (L1)]"
value,layer-1,ethereum,"[Smart Contract Platform, Layer 1 (L1), Ethere..."
value,layer-1,binancecoin,"[Alleged SEC Securities, Smart Contract Platfo..."


##### 1.2.1.3. Disponibilidad de informacion historica por categoria

In [8]:
list_coin_slctd = list(data_coins_markets['coin_id'].unique())
list_coin_slctd

['bitcoin',
 'ethereum',
 'binancecoin',
 'cardano',
 'solana',
 'polkadot',
 'the-open-network',
 'bitcoin-cash',
 'avalanche-2',
 'cosmos',
 'hedera-hashgraph',
 'crypto-com-chain',
 'near',
 'kaspa',
 'algorand',
 'tezos',
 'elrond-erd-2',
 'injective-protocol',
 'fantom',
 'radix',
 'tether',
 'usd-coin',
 'dai',
 'binance-usd',
 'true-usd',
 'frax',
 'usdd',
 'paxos-standard',
 'pax-gold',
 'tether-gold',
 'first-digital-usd',
 'gemini-dollar',
 'liquity-usd',
 'tether-eurt',
 'eusd-27a558b0-8b5b-4225-a614-63539da936f4',
 'alchemix-usd',
 'stasis-eurs',
 'terrausd',
 'bilira',
 'bob']

In [9]:
# global df vacio
data_market=pd.DataFrame()

# parametros
vs_currency = "usd"
days = "max"
interval = "daily"

# primer loop: para cada coin_id en mi lista seleccionada list_coin_slctd
for coin_id in list_coin_slctd:
    
    # subloop df vacio + coin_id para faciltiar merge
    data_market_id=pd.DataFrame([coin_id], columns=['coin_id'])

    # coingecko tiene una limite maximo de 30 consultas por min.;
    # introducimos una pausa de 2.1 segundos para no superar el limite:
    # 60/2.1 = 28.57 consultas por min.
    time.sleep(2.1)
    
    # consulta
    response_market_id = cg.get_coin_market_chart_by_id(
        id=coin_id,
        vs_currency=vs_currency,
        days=days,
        interval=interval)
    
    # lista con keys de la consulta: ['prices', 'market_caps', 'total_volumes']
    response_keys = list(response_market_id.keys())
    
    # loop para extraer variables para cada key del diccionario
    for k in response_keys:
        market_id_k = pd.DataFrame(response_market_id[k]).rename(columns={0: 'timestamp', 1: k})
        market_id_k['coin_id'] = coin_id
        market_id_k['year'] = pd.to_datetime(market_id_k['timestamp'], unit='ms').dt.year
        market_id_k['month'] = pd.to_datetime(market_id_k['timestamp'], unit='ms').dt.month
        market_id_k['day'] = pd.to_datetime(market_id_k['timestamp'], unit='ms').dt.day
        
        # unimos al df individual para cada coin_id
        market_id_k_2join = market_id_k.drop(columns=['timestamp'])
        data_market_id = data_market_id.merge(market_id_k_2join)
    
    # agregamos al df global
    data_market=pd.concat([data_market,data_market_id])

In [10]:
# incluimos datos de categoria para cada coin_id
data_coins_info_cat = data_coins_info[['category_id','coin_id','categories']]
data_market=data_market.merge(data_coins_info_cat)
data_market.info()
data_market.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56643 entries, 0 to 56642
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   coin_id        56643 non-null  object 
 1   prices         56643 non-null  float64
 2   year           56643 non-null  int32  
 3   month          56643 non-null  int32  
 4   day            56643 non-null  int32  
 5   market_caps    56641 non-null  float64
 6   total_volumes  56643 non-null  float64
 7   category_id    56643 non-null  object 
 8   categories     56643 non-null  object 
dtypes: float64(3), int32(3), object(3)
memory usage: 3.2+ MB


,coin_id,prices,year,month,day,market_caps,total_volumes,category_id,categories
0,bitcoin,135.30,2013,4,28,1.500518e+09,0.0,layer-1,"[Cryptocurrency, Layer 1 (L1)]"
1,bitcoin,141.96,2013,4,29,1.575032e+09,0.0,layer-1,"[Cryptocurrency, Layer 1 (L1)]"
2,bitcoin,135.30,2013,4,30,1.501657e+09,0.0,layer-1,"[Cryptocurrency, Layer 1 (L1)]"
3,bitcoin,117.00,2013,5,1,1.298952e+09,0.0,layer-1,"[Cryptocurrency, Layer 1 (L1)]"
4,bitcoin,103.43,2013,5,2,1.148668e+09,0.0,layer-1,"[Cryptocurrency, Layer 1 (L1)]"


In [11]:
# identificamos numero de observaciones por categoria y año
data_market_obs = data_market.groupby(['category_id','year'])['coin_id'].nunique()
data_market_obs.reset_index()

,category_id,year,coin_id
0,layer-1,2013,1
1,layer-1,2014,1
2,layer-1,2015,2
3,layer-1,2016,2
4,layer-1,2017,5
5,layer-1,2018,7
6,layer-1,2019,11
7,layer-1,2020,17
8,layer-1,2021,19
9,layer-1,2022,20


In [12]:
data_market_casos_layer1 = data_market.loc[
    (data_market['category_id'].str.contains('layer-1')) &
    (data_market['year']==2019)
    ].groupby(['category_id','year'])['coin_id'].unique().reset_index()

casos_layer1=list(data_market_casos_layer1['coin_id'].explode())
print("Numero de casos en categoria layer-1:",len(casos_layer1))

data_market_casos_stablecoins = data_market.loc[
    (data_market['category_id'].str.contains('stablecoins')) &
    (data_market['year']==2019)
    ].groupby(['category_id','year'])['coin_id'].unique().reset_index()

casos_stablecoins=list(data_market_casos_stablecoins['coin_id'].explode())
print("Numero de casos en categoria stablecoins:",len(casos_stablecoins))

Numero de casos en categoria layer-1: 11
Numero de casos en categoria stablecoins: 10


In [13]:
data_market_slctd=data_market.loc[
    (data_market['year'].isin(range(2019,2023+1))) &
    (((data_market['category_id'].str.contains('layer-1')) &
    (data_market['coin_id'].isin(casos_layer1))) |
    ((data_market['category_id'].str.contains('stablecoins')) &
    (data_market['coin_id'].isin(casos_stablecoins))))
]
data_market_slctd

,coin_id,prices,year,month,day,market_caps,total_volumes,category_id,categories
2072,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]"
2073,bitcoin,3794.264254,2019,1,2,6.618845e+10,2.689878e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]"
2074,bitcoin,3872.172184,2019,1,3,6.764330e+10,3.263469e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]"
2075,bitcoin,3776.525808,2019,1,4,6.592229e+10,2.261200e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]"
2076,bitcoin,3815.268080,2019,1,5,6.665077e+10,2.950055e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]"
...,...,...,...,...,...,...,...,...,...
56284,bilira,0.037038,2023,8,25,9.833678e+07,3.449758e+04,stablecoins,"[Finance / Banking, Stablecoins, BNB Chain Eco..."
56285,bilira,0.036606,2023,8,25,9.932376e+07,2.076894e+04,stablecoins,"[Finance / Banking, Stablecoins, BNB Chain Eco..."
56286,bilira,0.036606,2023,8,25,9.932376e+07,3.449758e+04,stablecoins,"[Finance / Banking, Stablecoins, BNB Chain Eco..."
56287,bilira,0.036606,2023,8,25,9.833678e+07,2.076894e+04,stablecoins,"[Finance / Banking, Stablecoins, BNB Chain Eco..."


In [14]:
# confirmamos numero de observaciones por categoria y año
data_market_slctd_obs = data_market_slctd.groupby(['category_id','year'])['coin_id'].nunique()
data_market_slctd_obs.reset_index()

,category_id,year,coin_id
0,layer-1,2019,11
1,layer-1,2020,11
2,layer-1,2021,11
3,layer-1,2022,11
4,layer-1,2023,11
5,stablecoins,2019,10
6,stablecoins,2020,10
7,stablecoins,2021,10
8,stablecoins,2022,10
9,stablecoins,2023,10


In [15]:
# exportamos el df final para alimentar el analisis
data_market_slctd.to_pickle(os.path.join("1_data","data_market.pkl"))
print(data_market['coin_id'].unique())
data_market_slctd.info()
data_market_slctd.head()

['bitcoin' 'ethereum' 'binancecoin' 'cardano' 'solana' 'polkadot'
 'the-open-network' 'bitcoin-cash' 'avalanche-2' 'cosmos'
 'hedera-hashgraph' 'crypto-com-chain' 'near' 'kaspa' 'algorand' 'tezos'
 'elrond-erd-2' 'injective-protocol' 'fantom' 'radix' 'tether' 'usd-coin'
 'dai' 'binance-usd' 'true-usd' 'frax' 'usdd' 'paxos-standard' 'pax-gold'
 'tether-gold' 'first-digital-usd' 'gemini-dollar' 'liquity-usd'
 'tether-eurt' 'eusd-27a558b0-8b5b-4225-a614-63539da936f4' 'alchemix-usd'
 'stasis-eurs' 'terrausd' 'bilira' 'bob']
<class 'pandas.core.frame.DataFrame'>
Index: 34099 entries, 2072 to 56288
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   coin_id        34099 non-null  object 
 1   prices         34099 non-null  float64
 2   year           34099 non-null  int32  
 3   month          34099 non-null  int32  
 4   day            34099 non-null  int32  
 5   market_caps    34099 non-null  float64
 6   total_volum

,coin_id,prices,year,month,day,market_caps,total_volumes,category_id,categories
2072,bitcoin,3692.531566,2019,1,1,6.442264e+10,2.991428e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]"
2073,bitcoin,3794.264254,2019,1,2,6.618845e+10,2.689878e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]"
2074,bitcoin,3872.172184,2019,1,3,6.764330e+10,3.263469e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]"
2075,bitcoin,3776.525808,2019,1,4,6.592229e+10,2.261200e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]"
2076,bitcoin,3815.268080,2019,1,5,6.665077e+10,2.950055e+09,layer-1,"[Cryptocurrency, Layer 1 (L1)]"


## 1.3. Extraccion de datos sobre paises

Como mencionamos anteriormente, nos interesa conocer la volatilidad desde el punto de vista de un contexto latinoamericano con niveles locales de inflacion considerable.

Nos concentramos en los cuatro paises mas grandes por poblacion de Latino-America: 'Brazil', 'Mexico', 'Argentina', 'Colombia'. Como referencia a esta monedas locales, tambien incluimos EEUU (USD).

Primero, extraemos datos de series de tiempo del PIB en precios corriente y constantes con el fin de generar indices de precios.

Segundo, extraemos datos de factor de conversion para expresar precios constantes en precios ajustados segun la paridad de poder adquisitivo. Para lograr esto, seguimos la metodologia desarrollada por el Banco Mundial (WB) y disponible en https://www.worldbank.org/en/programs/icp/brief/VC_Tech_1.


In [16]:
# exploramos las series de tiempo del PIB (gdp) disponibles en las bases de datos del WB
print(wb.series.info(q='gdp'))

id                    value
--------------------  -------------------------------------------------------------------------
EG.GDP.PUSE.KO.PP     GDP per unit of energy use (PPP $ per kg of oil equivalent)
EG.GDP.PUSE.KO.PP.KD  GDP per unit of energy use (constant 2017 PPP $ per kg of oil equivalent)
EG.USE.COMM.GD.PP.KD  Energy use (kg of oil equivalent) per $1,000 GDP (constant 2017 PPP)
NY.GDP.DEFL.KD.ZG     Inflation, GDP deflator (annual %)
NY.GDP.DEFL.KD.ZG.AD  Inflation, GDP deflator: linked series (annual %)
NY.GDP.DEFL.ZS        GDP deflator (base year varies by country)
NY.GDP.DEFL.ZS.AD     GDP deflator: linked series (base year varies by country)
NY.GDP.DISC.CN        Discrepancy in expenditure estimate of GDP (current LCU)
NY.GDP.DISC.KN        Discrepancy in expenditure estimate of GDP (constant LCU)
NY.GDP.MKTP.CD        GDP (current US$)
NY.GDP.MKTP.CN        GDP (current LCU)
NY.GDP.MKTP.CN.AD     GDP: linked series (current LCU)
NY.GDP.MKTP.KD        GDP (constant 201

In [17]:
# seleccionamos las series de interes
wb_gdp_cnstnt2015='NY.GDP.MKTP.KD' # GDP (constant 2015 US$)
wb_gdp_crrnt='NY.GDP.MKTP.CD' # GDP (current US$)
#print(wb.series.metadata.get(wb_gdp_cnstnt2015))
#print(wb.series.metadata.get(wb_gdp_crrnt))

In [18]:
codes_paises = wb.economy.coder(['Argentina', 'Brazil', 'Colombia', 'Mexico', 'USA'])
print(codes_paises)
list_codes_paises = list(pd.DataFrame(codes_paises, index=[0]).iloc[0])
print(list_codes_paises)

ORIGINAL NAME    WBG NAME       ISO_CODE
---------------  -------------  ----------
Argentina        Argentina      ARG
Brazil           Brazil         BRA
Colombia         Colombia       COL
Mexico           Mexico         MEX
USA              United States  USA
['ARG', 'BRA', 'COL', 'MEX', 'USA']


In [19]:
# extraemos data para wb_gdp_cnstnt2015
data_gdp_cnstnt2015 = wb.data.DataFrame(
    [wb_gdp_cnstnt2015],
    list_codes_paises,
    range(2015,2023)
    ).reset_index().rename(columns={'economy':'code_pais'}).melt(
        id_vars='code_pais',
        var_name='year',
        value_name='gdp_cnstnt2015')
data_gdp_cnstnt2015['year'] = data_gdp_cnstnt2015['year'].str[2:].astype(int)

# extraemos data para data_gdp_crrnt
data_gdp_crrnt = wb.data.DataFrame(
    [wb_gdp_crrnt],
    list_codes_paises,
    range(2015,2023)
    ).reset_index().rename(columns={'economy':'code_pais'}).melt(
        id_vars='code_pais',
        var_name='year',
        value_name='gdp_crrnt')
data_gdp_crrnt['year'] = data_gdp_crrnt['year'].str[2:].astype(int)

data_gdpdfltr = pd.merge(
    data_gdp_cnstnt2015,
    data_gdp_crrnt,
    on=['code_pais','year']
)

data_gdpdfltr.info()
data_gdpdfltr.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   code_pais       40 non-null     object 
 1   year            40 non-null     int64  
 2   gdp_cnstnt2015  40 non-null     float64
 3   gdp_crrnt       40 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.4+ KB


,code_pais,year,gdp_cnstnt2015,gdp_crrnt
0,ARG,2015,5.947493e+11,5.947493e+11
1,BRA,2015,1.802212e+12,1.802212e+12
2,COL,2015,2.934924e+11,2.934924e+11
3,MEX,2015,1.171870e+12,1.171870e+12
4,USA,2015,1.820602e+13,1.820602e+13


In [20]:
# producimos un indice de precios constantes con base en las series 'gdp_crrnt' y 'gdp_cnstnt2015'
data_gdpdfltr['gdpdfltr_2015']=(data_gdpdfltr['gdp_crrnt']/data_gdpdfltr['gdp_cnstnt2015'])*100
data_gdpdfltr.info()
data_gdpdfltr.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   code_pais       40 non-null     object 
 1   year            40 non-null     int64  
 2   gdp_cnstnt2015  40 non-null     float64
 3   gdp_crrnt       40 non-null     float64
 4   gdpdfltr_2015   40 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 1.7+ KB


,code_pais,year,gdp_cnstnt2015,gdp_crrnt,gdpdfltr_2015
0,ARG,2015,5.947493e+11,5.947493e+11,100.000000
1,BRA,2015,1.802212e+12,1.802212e+12,100.000000
2,COL,2015,2.934924e+11,2.934924e+11,100.000000
3,MEX,2015,1.171870e+12,1.171870e+12,100.000000
4,USA,2015,1.820602e+13,1.820602e+13,100.000000
5,ARG,2016,5.823766e+11,5.575323e+11,95.733992
6,BRA,2016,1.743173e+12,1.795693e+12,103.012910
7,COL,2016,2.996187e+11,2.827201e+11,94.359972
8,MEX,2016,1.202696e+12,1.078493e+12,89.672919
9,USA,2016,1.850960e+13,1.869511e+13,101.002235


In [21]:
# exploramos las series de tiempo expresadas en ppp
print(wb.series.info(q='PPP conversion factor'))

id              value
--------------  ------------------------------------------------------------------------
PA.NUS.PPP      PPP conversion factor, GDP (LCU per international $)
PA.NUS.PPPC.RF  Price level ratio of PPP conversion factor (GDP) to market exchange rate
PA.NUS.PRVT.PP  PPP conversion factor, private consumption (LCU per international $)
                3 elements


In [2]:
# seleccionamos las series de interes
wb_pppfactor='PA.NUS.PPPC.RF' # Price level ratio of PPP conversion factor (GDP) to market exchange rate
print(wb.series.metadata.get(wb_pppfactor))

Series: PA.NUS.PPPC.RF

IndicatorName: Price level ratio of PPP conversion factor (GDP) to market exchange rate
--------
License_Type: CC BY-4.0
--------
License_URL: https://datacatalog.worldbank.org/public-licenses#cc-by
--------
Longdefinition: Price level ratio is the ratio of a purchasing power parity (PPP) conversion factor to an exchange rate. It provides a measure of the differences in price levels between countries by indicating the number of units of the common currency needed to buy the same volume of the aggregation level in each country. At the level of GDP, they provide a measure of the differences in the general price levels of countries.
--------
Periodicity: Annual
--------
Source: International Comparison Program, World Bank | World Development Indicators database, World Bank | Eurostat-OECD PPP Programme.
--------
Statisticalconceptandmethodology: For more information on underlying GDP in current international dollar, please refer to the metadata for "GDP, PPP (curre

In [23]:
# extraemos data para wb_pppfactor
data_pppfactor = wb.data.DataFrame(
    [wb_pppfactor],
    list_codes_paises,
    range(2015,2023) # solo necesitamos el PPP del año base del nuestro indice de precio
    ).reset_index().rename(columns={'economy':'code_pais'}).melt(
        id_vars='code_pais',
        var_name='year',
        value_name='pppfactor')
data_pppfactor['year'] = data_pppfactor['year'].str[2:].astype(int)
data_pppfactor.info()
data_pppfactor.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   code_pais  40 non-null     object 
 1   year       40 non-null     int64  
 2   pppfactor  40 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ KB


,code_pais,year,pppfactor
0,ARG,2015,0.685846
1,BRA,2015,0.597797
2,COL,2015,0.465566
3,MEX,2015,0.525458
4,USA,2015,1.000000
5,ARG,2016,0.629818
6,BRA,2016,0.610968
7,COL,2016,0.424888
8,MEX,2016,0.452530
9,USA,2016,1.000000


In [24]:
# logica de operacion
pppfactor_2join = data_pppfactor.loc[
    data_pppfactor['year']==2015][
        ['code_pais','year','pppfactor']].rename(
            columns={
                'year':'base_year',
                'pppfactor':'pppfactor_2015'}).copy()
data_pppfactor_check=pd.merge(
    data_gdpdfltr,
    data_pppfactor
).merge(pppfactor_2join)
data_pppfactor_check['gdp_crrntppp']=data_pppfactor_check['gdp_crrnt']/data_pppfactor_check['pppfactor']
data_pppfactor_check['gdp_ppp2015']=data_pppfactor_check['gdp_cnstnt2015']/data_pppfactor_check['pppfactor_2015']
data_pppfactor_check.head(10)

,code_pais,year,gdp_cnstnt2015,gdp_crrnt,gdpdfltr_2015,pppfactor,base_year,pppfactor_2015,gdp_crrntppp,gdp_ppp2015
0,ARG,2015,5.947493e+11,5.947493e+11,100.000000,0.685846,2015,0.685846,8.671768e+11,8.671768e+11
1,ARG,2016,5.823766e+11,5.575323e+11,95.733992,0.629818,2015,0.685846,8.852275e+11,8.491366e+11
2,ARG,2017,5.987909e+11,6.436284e+11,107.488014,0.619272,2015,0.685846,1.039331e+12,8.730696e+11
3,ARG,2018,5.831181e+11,5.248199e+11,90.002331,0.506358,2015,0.685846,1.036459e+12,8.502179e+11
4,ARG,2019,5.714507e+11,4.477547e+11,78.354031,0.433056,2015,0.685846,1.033943e+12,8.332062e+11
5,ARG,2020,5.146300e+11,3.855404e+11,74.916031,0.408721,2015,0.685846,9.432857e+11,7.503586e+11
6,ARG,2021,5.681426e+11,4.872271e+11,85.757899,0.447754,2015,0.685846,1.088158e+12,8.283827e+11
7,ARG,2022,5.979305e+11,6.327703e+11,105.826723,0.516364,2015,0.685846,1.225435e+12,8.718152e+11
8,BRA,2015,1.802212e+12,1.802212e+12,100.000000,0.597797,2015,0.597797,3.014755e+12,3.014755e+12
9,BRA,2016,1.743173e+12,1.795693e+12,103.012910,0.610968,2015,0.597797,2.939094e+12,2.915994e+12


In [25]:
# replicamos
pppfactor_2join = data_pppfactor.loc[
    data_pppfactor['year']==2015][
        ['code_pais','year','pppfactor']].rename(
            columns={
                'year':'base_year',
                'pppfactor':'pppfactor_2015'}).copy()
# unimos
data_deflators=pd.merge(
    data_gdpdfltr,
    data_pppfactor
).merge(pppfactor_2join)

# exportamos
data_deflators_export=data_deflators.loc[:,
    ['code_pais','year','gdpdfltr_2015','pppfactor_2015']
].copy()
data_deflators_export.to_pickle(os.path.join("1_data","data_deflators.pkl"))
data_deflators_export.info()
data_deflators_export.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   code_pais       40 non-null     object 
 1   year            40 non-null     int64  
 2   gdpdfltr_2015   40 non-null     float64
 3   pppfactor_2015  40 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.4+ KB


,code_pais,year,gdpdfltr_2015,pppfactor_2015
0,ARG,2015,100.000000,0.685846
1,ARG,2016,95.733992,0.685846
2,ARG,2017,107.488014,0.685846
3,ARG,2018,90.002331,0.685846
4,ARG,2019,78.354031,0.685846
